In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import tensorflow as tf
import keras
from PIL import Image,ImageTk
import keras.backend as K
import numpy as np
from keras.layers import Input, Dense, Activation, ZeroPadding2D, Flatten, Conv2D
from keras.layers import MaxPooling2D
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.preprocessing import image as image1
from keras.models import load_model
K.set_image_data_format('channels_last')
from matplotlib.pyplot import imshow
import os
class system:
    classes=[]
    @staticmethod
    def pre_process(fname):
        if (fname == 'b'):
            return [1, 0, 0, 0]
        elif (fname == 'is'):
            return [0, 1, 0, 0]
        elif (fname == 'iv'):
            return [0, 0, 1, 0]
        else:
            return [0, 0, 0, 1]
    @staticmethod
    def cropimages(img):
        z = np.asarray(img, dtype=np.int8)
        c = []
        for i in range(3):
            for j in range(4):
                crop = z[512 * i:512 * (i + 1), 512 * j:512 * (j + 1), :]
                c.append(crop)
        return c
    @staticmethod
    def uploadimages(path):
        x = []
        y = []
        cnt = 0
        for foldname in os.listdir(path):
            for filename in os.listdir(os.path.join(path, foldname)):
                img = Image.open(os.path.join(os.path.join(path, foldname), filename))
                crpImgs = system.cropimages(img)
                cnt += 1
                if cnt % 10 == 0:
                    print(str(cnt) + " Images are loading")
                for im in crpImgs:
                    x.append(np.divide(np.asarray(im, np.float16), 255.))
                    y.append(pre_process(foldname))
        print("Images are  cropped")
        print("converting images to array")
        return x, y, cnt
    @staticmethod
    def loadingdatsets(testNum=2):
        print("Loading data set images..")
        train_set_x_orig, train_set_y_orig, cnt = uploadimages(dataTrainPath)
        testNum = 24
        trainNum = (cnt * 12) - testNum
        print(testNum, trainNum)
        train_set_x_orig = np.array(train_set_x_orig, np.float16)
        train_set_y_orig = np.array(train_set_y_orig, np.int8)
        nshapeX = train_set_x_orig.shape
        nshapeY = train_set_y_orig.shape
        print("folder trainX" + str(nshapeX))
        print("folder trainY" + str(nshapeY))
        print("Images loaded")
        print("Loading all data")
        test_set_x_orig = train_set_x_orig[trainNum:, :, :, :]
        train_set_x_orig = train_set_x_orig[0:trainNum, :, :, :]
        test_set_y_orig = train_set_y_orig[trainNum:]
        train_set_y_orig = train_set_y_orig[0:trainNum]
        classes = np.array(os.listdir(dataTrainPath))  
        print(train_set_y_orig[0:50, :])
        print(train_set_x_orig[1])
        print("Data load complete")
        return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes
    @staticmethod
    def definingmodel(input_shape):
        X_input = Input(input_shape)
        X = Conv2D(16, (3, 3), strides=(1, 1))(X_input) 
        X = Activation('relu')(X)
        X = MaxPooling2D((3, 3), strides=3)(X)  
        X = Conv2D(32, (3, 3), strides=(1, 1))(X)
        X = Activation('relu')(X)
        X = MaxPooling2D((2, 2), strides=2)(X) 
        X = Conv2D(64, (2, 2), strides=(1, 1))(X)  
        X = Activation('relu')(X)
        X = ZeroPadding2D(padding=(2, 2))(X)  
        X = MaxPooling2D((2, 2), strides=2)(X)  
        X = Conv2D(64, (2, 2), strides=(1, 1))(X) 
        X = Activation('relu')(X)
        X = ZeroPadding2D(padding=(2, 2))(X)  
        X = MaxPooling2D((3, 3), strides=3)(X)  
        X = Conv2D(32, (3, 3), strides=(1, 1))(X)  
        X = Activation('relu')(X)
        X = Flatten()(X)  # Convert it to FC
        X = Dense(256, activation='relu')(X)  
        X = Dense(128, activation='relu')(X) 
        X = Dense(4, activation='softmax')(X)
        model = Model(inputs=X_input, outputs=X, name='Model')
        return model
    @staticmethod
    def train(batch_size, epochs):
        config = tf.compat.v1.ConfigProto()
        sess = tf.compat.v1.Session(config=config)
        tf.compat.v1.keras.backend.set_session(sess)
        model = definingmodel(X_train.shape[1:])
        model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
        while True:
            try:
                model = load_model('my_model3.h5')
            except:
                print("Training a new model")
            model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size) 
            model.save('my_model3.h5')
            preds = model.evaluate(X_test, Y_test_orig, batch_size=1, verbose=1, sample_weight=None)
            print(preds)
            print()
            print("Loss = " + str(preds[0]))
            print("Test Accuracy = " + str(preds[1]) + "\n\n\n\n\n")
            ch = input("Do you wish to continue training? (y/n) ")
            if ch == 'y':
                epochs = int(input("How many epochs this time? : "))
                continue
            else:
                break
        return model
    @staticmethod
    def predict(img, savedModelPath, showImg=False):
        model = load_model(savedModelPath)
        x = img
        Image.fromarray(np.array(img, np.float16), 'RGB')
        x = np.expand_dims(x, axis=0)

        softMaxPred = model.predict(x)

        probs = system.softmaxToProbs(softMaxPred)
        maxprob = 0
        maxI = 0
        for j in range(len(probs)):
            if probs[j] > maxprob:
                maxprob = probs[j]
                maxI = j
        return maxI, probs
    @staticmethod
    def softmaxToProbs(soft):
        z_exp = [np.math.exp(i) for i in soft[0]]
        sum_z_exp = sum(z_exp)
        return [(i / sum_z_exp) * 100 for i in z_exp]
    @staticmethod
    def predictImage(img_path='my_image.jpg', arrayImg=None, printData=False):
        crops = []
        if arrayImg == None:
            img = image1.load_img(img_path)
            crops = np.array(system.cropimages(img), np.float16)
            crops = np.divide(crops, 255.)
        Image.fromarray(np.array(crops[0]), "RGB")
        classes = []
        classes.append("Benign")
        classes.append("InSitu")
        classes.append("Invasive")
        classes.append("Normal")
        compProbs = [0,0,0,0]
        for i in range(len(crops)):
            if printData:
                print("\n\nCrop " + str(i + 1) + " prediction:\n")
            _, probs = system.predict(crops[i], 'my_model3.h5', showImg=False)
            for j in range(len(classes)):
                if printData:
                    print(str(classes[j]) + " : " + str(round(probs[j], 4)) + "%")
                compProbs[j] += probs[j]
        root.destroy()
        root1=Tk()
        root1.title("cancer results")
        width, height = root1.winfo_screenwidth(), root1.winfo_screenheight()
        root1.geometry('%dx%d+0+0' % (width,height))
        image = Image.open('Insitu1.jpg')
        image = image.resize((500,400), Image.ANTIALIAS)
        my_img = ImageTk.PhotoImage(image)
        my_lbl = Label(root1,image = my_img)
        my_lbl.place(relx=0.2,rely=0.28)
        Label(root1,text="REPORT",font=("Arial",30)).place(relx=0.5,rely=0.1)
        Label(root1,text="Cancer Test Results\n" ,font=("Arial",20)).place(relx=0.6,rely=0.3)
        Label(root1,text=str(classes[0]) + " : " + str(round(compProbs[0] / 12, 4)) + "%" ,font=("Arial",20)).place(relx=0.6,rely=0.4)
        Label(root1,text=str(classes[1]) + " : " + str(round(compProbs[1] / 12, 4)) + "%" ,font=("Arial",20)).place(relx=0.6,rely=0.5)
        Label(root1,text=str(classes[2]) + " : " + str(round(compProbs[2] / 12, 4)) + "%" ,font=("Arial",20)).place(relx=0.6,rely=0.6)
        Label(root1,text=str(classes[3]) + " : " + str(round(compProbs[3] / 12, 4)) + "%",font=("Arial",20)).place(relx=0.6,rely=0.7)
        Label(root1,text="The type of cancer detected is"+" : "+classes[_],font=("Arial",25)).place(relx=0.3,rely=0.82)
        root1.mainloop()
class user:
    @staticmethod
    def cancer_detection():
        ch =2
        if ch == 1:
            try:
                classes = np.load('classes.npy')
                print("Loading")
                X_train = np.load('X_train.npy')
                Y_train = np.load('Y_train.npy')
                X_test = np.load('X_test.npy')
                Y_test_orig = np.load('Y_test_orig.npy')
            except:
                X_train, Y_train, X_test, Y_test_orig, classes = system.loadingdatsets()
                print("Saving...")
                np.save('X_train', X_train)
                np.save('Y_train', Y_train)
                np.save('X_test', X_test)
                np.save('Y_test_orig', Y_test_orig)
                np.save('classes', classes)

            print("number of training examples = " + str(X_train.shape[0]))
            print("number of test examples = " + str(X_test.shape[0]))
            print("X_train shape: " + str(X_train.shape))
            print("Y_train shape: " + str(Y_train.shape))
            print("X_test shape: " + str(X_test.shape))
            print("Y_test shape: " + str(Y_test_orig.shape))
            model = system.train(batch_size=100, epochs=1)

        elif ch == 2:
                bot=tempdir.name
                system.predictImage(img_path=bot)
        else:
            print("Please enter only 1 or 2")
from tkinter import Label,Entry,Button,Tk,filedialog,PhotoImage,Canvas,Toplevel
root=Tk()
root.title("cancer")
width, height = root.winfo_screenwidth(), root.winfo_screenheight()
root.geometry('%dx%d+0+0' % (width,height))
image = Image.open('nadhi.jpg')
image = image.resize((width, height), Image.ANTIALIAS)
my_img = ImageTk.PhotoImage(image)
my_lbl = Label(image = my_img)
my_lbl.place(relx=0,rely=0)
Label(root,text="Cancer Detection Using Convoluton Neural Networks",bg='salmon2',font=("Arial",30)).place(relx=0.2,rely=0.3)
Label(root,text="Click on Browse To Choose Filename",bg='salmon2',font=("Arial",25)).place(relx=0.2,rely=0.5)
Button(root, text = 'browse',bg='ivory2',fg='black',font=("Arial",25),command=lambda:[upload_image(),user.cancer_detection()]).place(relx=0.6,rely=0.489)
tempdir=''
def upload_image ():
    global tempdir
    currdir = os.getcwd()
    tempdir = filedialog.askopenfile(parent=root,initialdir="/")

root.mainloop()

Using TensorFlow backend.


In [1]:
import tkinter
from tkinter import 
import os

root = tkinter.Tk()
root.withdraw() #use to hide tkinter window

def search_for_file_path ():
    currdir = os.getcwd()
    tempdir = filedialog.askdirectory(parent=root, initialdir=currdir, title='Please select a directory')
    if len(tempdir) > 0:
        print ("You chose: %s" % tempdir)
    return tempdir


file_path_variable = search_for_file_path()
print (file_path_variable)

SyntaxError: invalid syntax (<ipython-input-1-dfbaea1867ea>, line 2)